# San Francisco city employee salary data

<br>
<img src="img/san_francisco.png">
<br>

Um dos conjuntos de dados mais populares que as cidades normalmente lançam é sua estrutura salarial.

Foi processado e enviado o conjunto de dados de salários do SF para entender melhor como São Francisco paga a seus funcionários e o que isso pode nos informar sobre como a sede da cidade está sendo adminstrada.

Inicilamente, pede-se a média salarial das 10 profissões com maior incidência no relatório, considerando o ano de 2014, com uma visualização de sua distribuição.

Para download dos dados acesse [2011–2018 salaries for San Francisco](https://transparentcalifornia.com/salaries/san-francisco/)

Alguns ângulos potencialmente interessantes de exploração:

- Quantos trabalhadores ganham acima e abaixo da média na função de Deputy Sheriff?
- Das 200 profissões com maior incidência, qual delas tem média de ganhos acima e abaixo da média do salário de Seputy Sheriff?

Levante algumas hipóteses a sua escolha e comprove se é possível não-rejeitar ou rejeitar, respondendo se as médias são estatisticamente iguais/equivalentes ou estatisticamente diferentes/significativas.

In [ ]:
# importação das bibliotecas
import pandas as pd
import numpy as np

import scipy
from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# ler e armazenar os dados

#%time salary = pd.read_csv("http://www.dropbox.com/s/as9lqe6u9fuxn5t/SanFrancisco_salaries.csv?dl=1",sep=',',encoding= 'latin1')

In [ ]:
%time salary = pd.read_csv(r'data/SanFrancisco_salaries.csv')

In [ ]:
salary.info()

In [ ]:
# renomear as colunas do DataFrame sem espaços
salary.rename(columns={'Employee Name':'EmployeeName',
                       'Job Title':'JobTitle',
                       'Base Pay':'BasePay', 
                       'Overtime Pay':'OvertimePay',
                       'Other Pay':'OtherPay',
                       'Total Pay':'TotalPay',
                       'Total Pay & Benefits':'TotalPayBenefits'},
                       inplace=True)

## 1. Cálculo da média dos salários das dez maiores ocupações do DataFrame

In [ ]:
# contagem percentual dos dez maiores ocupações do DataFrame
salary.JobTitle.value_counts(3).nlargest(10)

In [ ]:
# armazenar a série das dez maiores ocupações em uma vairável jobs
jobs = salary.JobTitle.value_counts(3).nlargest(10).index
jobs

In [ ]:
# cálculo da média dos salários das dez maiores ocupações 

keys = []
values = []
salaries_list = []

for job in jobs:
    keys.append(job)
    values.append(round(salary[salary['JobTitle'] == job]['TotalPay'].mean(),2))
    
salaries_list = list(zip(keys, values))
salaries_list

In [ ]:
# armanzenando os salários das dez maiores ocupações em um DataFrame
df_salaries = pd.DataFrame(salaries_list,columns=['profession','salary'])
df_salaries.sort_values(by=['salary'],ascending=False)

In [ ]:
# plotando as distribuições dos salários das dez maiores ocupações

from matplotlib import pyplot
%matplotlib inline

bins = 200

from matplotlib.pyplot import figure
figure(figsize=(15, 10), dpi=80)

for j in jobs:
    ax = pyplot.hist(salary[salary['JobTitle'] == j]['TotalPay'], bins, alpha=0.5, label=j, range=(0,300000))

pyplot.legend(loc='upper right')
pyplot.show()

## 2. Quantos trabalhadores ganham acima e abaixo da média na função de Deputy Sheriff?


In [ ]:
# inofrmações básicas do DataFrame
salary.info()

In [ ]:
# Plotagem da distribuição dos salários da ocupação "Deputy Sheriff"
salary[salary.JobTitle == 'Deputy Sheriff']['TotalPay'].hist(bins=50)

In [ ]:
# cálculo da média e desvio-padrão da série de salários dos "Deputy Sheriff"

x = salary[salary.JobTitle == 'Deputy Sheriff']['TotalPay']

mu = np.mean(x)
std = np.std(x)
mu,std

In [ ]:
# cálculo da densidade de probabilidade para dois desvios-padrão
scipy.stats.norm(mu, std).pdf(2*std/(len(x)**0.5))

In [ ]:
# cálculo do intervalo de confiança para 95% de nível de confiança
scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)

Como a distribuição já tem um perfil normal, vamos padronizar os dados e fazer a inferência

In [ ]:
# definição de um DataFrmae somente com os salários de 'Deputy Sheriff'
sheriff = salary[salary.JobTitle == 'Deputy Sheriff']
sheriff.head()

In [ ]:
# padronizando os dados da série de salários de "Deputy Sheriff"
sheriff['norm_pay'] = (sheriff['TotalPay'] - sheriff['TotalPay'].mean()) / sheriff['TotalPay'].std()
sheriff['norm_pay'].hist(bins=50)

In [ ]:
# verificando o número de observações
len(sheriff['norm_pay'])

In [ ]:
# calculando a média e o desvio-padrão dos salários de 'Deputy Sheriff'
x = sheriff['norm_pay']

mu = np.mean(x)
std = np.std(x)
mu,std

In [ ]:
# Verificando o Z-score dos salários de 'Deputy Sheriff'

scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)

In [ ]:
# Verificando o p-values dos salários de 'Deputy Sheriff'

scipy.stats.norm(mu, std).pdf(2*std)

## Deputy Sheriff que ganham abaixo da média salarial

In [ ]:
# filtro dos "Deputy Sheriff" abaixo no salário médio com 95% de confiança
sheriff[sheriff['norm_pay']<scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[0]]

In [ ]:
len(sheriff[sheriff['norm_pay']<scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[0]])

### Deputy Sheriff que ganham acima da média salarial

In [ ]:
# filtro dos "Deputy Sheriff" acima no salário médio com 95% de confiança
sheriff[sheriff['norm_pay']>scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[1]]

In [ ]:
len(sheriff[sheriff['norm_pay']>scipy.stats.norm.interval(alpha=0.95,loc=mu,scale=std)[1]])

## Das 200 profissões com maior incidência, qual delas tem média de ganhos acima e abaixo da média do salário de Seputy Sheriff?

In [ ]:
# verificando todas as profissiões do DataSet
salary.JobTitle.unique()

In [ ]:
# normalizando todos os valores de salário
salary['norm_pay'] = (salary['TotalPay'] - salary['TotalPay'].mean()) / salary['TotalPay'].std()
salary['norm_pay'].hist(bins=50)

In [ ]:
# verificando os dados
salary.head()

In [ ]:
# selecionando as 200 profissões com maior inciência
salary.JobTitle.value_counts()[:200].index

In [ ]:
# verificando a inferência nos salários de Deputy Sheriff e IS Engineer-Senior
# utilizando o t-test para a média de duas séries

stats.ttest_ind(a=salary.norm_pay[salary.JobTitle == 'Deputy Sheriff'],
                b=salary.norm_pay[salary.JobTitle == 'IS Engineer-Senior'], 
                equal_var = False)

Como o p-value é menor que 0.05 podemos afirmar que a média de salários de Deputy Sheriff e Engineer Senior é estatísticamente diferente.

In [ ]:
# verificando a inferência nos salários de Deputy Sheriff e Sr. Stationary Eng, Sew Plant
stats.ttest_ind(a=salary.norm_pay[salary.JobTitle == 'Deputy Sheriff'],
                b=salary.norm_pay[salary.JobTitle == 'Sr. Stationary Eng, Sew Plant'], 
                equal_var = False)

Como o p-value é maior que 0.05 podemos afirmar que a média de salários de Deputy Sheriff e Engineer Senior é estatisticamente equivalente.

In [ ]:
# aplicando o t-test para médias de duas séries entre Deputy Sheriff e todos os JobTitles

jobtitle = []
pvalues = []
for i in salary.JobTitle.value_counts()[:200].index:
    jobtitle_tstats = stats.ttest_ind(a=salary.norm_pay[salary.JobTitle == 'Deputy Sheriff'],
                                      b=salary.norm_pay[salary.JobTitle == i], 
                                      equal_var = False)
    jobtitle.append(i)
    pvalues.append(jobtitle_tstats[1].round(4))
    
# colocando os valores em um DataFrame e listando
inference=pd.DataFrame(list(zip(jobtitle,pvalues)),columns =['jobtitle', 'pvalues'])
inference.sort_values(by=['pvalues'],ascending=False)

In [ ]:
# listando os valores que tem média de salário acima da média de Deputy Sheriff
list(inference.jobtitle[inference.pvalues > 0.05])

In [ ]:
# contando as profissões com média de salário acima da média do salário de Deputy Sheriff
inference.jobtitle[inference.pvalues > 0.05].count()

11 profissões tem média de remuneração acima de Deputy Sheriff com 95% de confiança.

In [ ]:
# listando os valores que tem média de salário abaixo da média de Deputy Sheriff
list(inference.jobtitle[inference.pvalues < 0.05])

In [ ]:
# contando as profissões com média de salário abaixo da média do salário de Deputy Sheriff
inference.jobtitle[inference.pvalues < 0.05].count()

189 profissões tem média de remuneração abaixo de Deputy Sheriff com 95% de confiança.